In [4]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd

import random
import string
import numpy as np

import sys, os

import torch.utils.data as data

os.environ["CUDA_VISIBLE_DEVICES"] = '0'

all_characters = string.printable
number_of_characters = len(all_characters)


def character_to_label(character):
    ''' Returns the index of the character in the dictionary'''
    
    character_label = all_characters.find(character)   
    return character_label

def string_to_labels(character_string):
    ''' Returns a list of indcies for each character in the word'''
    
    return map(lambda character: character_to_label(character), character_string)

def pad_sequence(seq, max_length, pad_label=100):
    ''' Pads the sequence to have sequences of equal length'''
    
    seq += [pad_label for i in range(max_length - len(seq))]
    return seq


class LyricsGenerationDataset(data.Dataset):
    ''' Creates the data to be fed into the RNN'''
    
    def __init__(self, csv_file_path, minimum_song_count=10, artists=["Tyagaraja"], part=None):
        
        
        self.lyrics_dataframe = pd.read_csv(csv_file_path)
        self.section = part
        
        if artists:    
            self.lyrics_dataframe = self.lyrics_dataframe[self.lyrics_dataframe.artist.isin(artists)]
            self.lyrics_dataframe = self.lyrics_dataframe.reset_index()
        
        # REMOVE
        if minimum_song_count:
            self.lyrics_dataframe = self.lyrics_dataframe.groupby('artist').filter(lambda x: len(x) > minimum_song_count)
            # Reindex .loc after we fetched random songs
            self.lyrics_dataframe = self.lyrics_dataframe.reset_index()
            
        self.lyrics_dataframe = self.lyrics_dataframe[part].dropna()
        self.lyrics_dataframe = self.lyrics_dataframe.reset_index()
        if part == "pallavi":
            self.max_text_len = self.lyrics_dataframe.pallavi.map(lambda x: len(x)).max()
        if part == "anupallavi":
            self.max_text_len = self.lyrics_dataframe.anupallavi.map(lambda x: len(x)).max()
        if part == "f_caranam":
            self.max_text_len = self.lyrics_dataframe.f_caranam.map(lambda x: len(x)).max()
        if part == "l_caranam":
            self.max_text_len = self.lyrics_dataframe.l_caranam.map(lambda x: len(x)).max()
            
        # Length of biggest lyrics to be used for padding
#         self.max_text_len = int(self.lyrics_dataframe.part.str.len().max())
        print ("Part:", part)
        print ("max_text_len", self.max_text_len)
        
        # Number of songs
        whole_dataset_len = len(self.lyrics_dataframe)
        
        # Create indices array of the size - number of songs
        self.indexes = range(whole_dataset_len)
        
        # List of artists in the csv file
#         self.artists_list = list(self.lyrics_dataframe.artist.unique())
        
        # Number of artists
#         self.number_of_artists = len(self.artists_list)
    
    
    def __len__(self):
        
        return len(self.indexes)
    
    
    def __getitem__(self, index):    # obj[i] basically calls get_item function
        ''' Returns a tuple of padded input_string, output_string and the sequence length'''   
        
        # Get the text of the song... Each sequence is a text of a single song
        index = self.indexes[index]
        if self.section == "pallavi":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].pallavi)
        if self.section == "anupallavi":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].anupallavi)
        if self.section == "f_caranam":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].f_caranam)
        if self.section == "l_caranam":
          sequence_raw_string = str(self.lyrics_dataframe.loc[index].l_caranam)
        # Map the song text to indices of the dictionary
        sequence_string_labels = string_to_labels(sequence_raw_string)
        sequence_length = len(sequence_string_labels) - 1
        
        input_string_labels = sequence_string_labels[:-1]       # All except the last character
        output_string_labels = sequence_string_labels[1:]       # All except the first character
                
        # Pad sequence to max text length
        input_string_labels_padded = pad_sequence(input_string_labels, max_length=self.max_text_len)
        output_string_labels_padded = pad_sequence(output_string_labels, max_length=self.max_text_len, pad_label=-100)
        
        return (torch.LongTensor(input_string_labels_padded),
                torch.LongTensor(output_string_labels_padded),
                torch.LongTensor([sequence_length]) )

    
def post_process_sequence_batch(batch_tuple):
    ''' Padding only upto the max size in the given batch'''
    
    input_sequences, output_sequences, lengths = batch_tuple
    
    # Creates a tuple of each element as a tensor
    splitted_input_sequence_batch = input_sequences.split(split_size=1)
    splitted_output_sequence_batch = output_sequences.split(split_size=1)
    splitted_lengths_batch = lengths.split(split_size=1)

    # Creates a list of each element being a tuple of input, output and batch_length
    training_data_tuples = zip(splitted_input_sequence_batch,
                               splitted_output_sequence_batch,
                               splitted_lengths_batch)

    # Sort them according to sequence length
    training_data_tuples_sorted = sorted(training_data_tuples,
                                         key=lambda p: int(p[2]),
                                         reverse=True)

    splitted_input_sequence_batch, splitted_output_sequence_batch, splitted_lengths_batch = zip(*training_data_tuples_sorted)

    # Concatenate each individual tensors into one tensor
    input_sequence_batch_sorted = torch.cat(splitted_input_sequence_batch)
    output_sequence_batch_sorted = torch.cat(splitted_output_sequence_batch)
    lengths_batch_sorted = torch.cat(splitted_lengths_batch)
    
    # Get the sequence padding only upto the max in the given batch
    input_sequence_batch_sorted = input_sequence_batch_sorted[:, :lengths_batch_sorted[0, 0]]
    output_sequence_batch_sorted = output_sequence_batch_sorted[:, :lengths_batch_sorted[0, 0]]
    input_sequence_batch_transposed = input_sequence_batch_sorted.transpose(0, 1)
    
    # pytorch's api for rnns needs lengths to be list of int
    lengths_batch_sorted_list = list(lengths_batch_sorted)
    lengths_batch_sorted_list = map(lambda x: int(x), lengths_batch_sorted_list)
    
    return input_sequence_batch_transposed, output_sequence_batch_sorted, lengths_batch_sorted_list


class RNN(nn.Module):
    ''' The Recurrent Neural Network Class'''
    
    def __init__(self, input_size, hidden_size, num_classes, n_layers=2):
        ''' Initializing the neural network with the hyper-parameters'''
        
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.n_layers = n_layers
        
	     	# Instead of creating one-hot encoding which would be too big...
		    # we use embedding which creates a fixed size encoding -> input size(vocab size), output size (encoding size) 
        self.encoder = nn.Embedding(input_size, hidden_size)
        
        # Initializing the LSTM layers
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        
        # Creating a fully connected layer to finally give output with the number of classes we want it to have
        self.logits_fc = nn.Linear(hidden_size, num_classes)
    
    
    def forward(self, input_sequences, input_sequences_lengths, hidden=None):
        
        batch_size = input_sequences.shape[1]
        embedded = self.encoder(input_sequences)

		    # Packing efficiently handles batched sequences with variable lengths i.e the unpadded regions
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_sequences_lengths)
        # Get output and hidden to be used for the next hidden
        outputs, hidden = self.lstm(packed, hidden)
        # Unpack the data into sequences with the padding
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # fully connected layer to get the output into required number of classes
        logits = self.logits_fc(outputs)
        logits = logits.transpose(0, 1).contiguous()   # Data should be contiguous in memory for the loss function
        logits_flatten = logits.view(-1, self.num_classes)
        
        return logits_flatten, hidden


# Getting the input from the CSV files for pallavi, anupallavi, caranam and last caranams
p_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='pallavi')
ap_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='anupallavi')
fc_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='f_caranam')
lc_trainset = LyricsGenerationDataset(csv_file_path='div_songs.csv', part='l_caranam')

p_trainset_loader = torch.utils.data.DataLoader(p_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
ap_trainset_loader = torch.utils.data.DataLoader(ap_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
fc_trainset_loader = torch.utils.data.DataLoader(fc_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
lc_trainset_loader = torch.utils.data.DataLoader(lc_trainset, batch_size=50, shuffle=True, num_workers=4, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size

data_frame = pd.read_csv('div_songs.csv')['no_caranam']
caranam_mean = np.mean(data_frame)
caranam_std = np.std(data_frame)
data_frame = pd.read_csv('div_songs.csv')['no_ap']
ap_mean = np.mean(data_frame)
ap_std = np.std(data_frame)
# p_trainset_loader = torch.utils.data.DataLoader(p_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
# ap_trainset_loader = torch.utils.data.DataLoader(ap_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
# fc_trainset_loader = torch.utils.data.DataLoader(fc_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size
# lc_trainset_loader = torch.utils.data.DataLoader(lc_trainset, batch_size=50, num_workers=1, drop_last=True)    # drop the last batch if the dataset size is not divisible by the batch_size

('Part:', 'pallavi')
('max_text_len', 158)
('Part:', 'anupallavi')
('max_text_len', 177)
('Part:', 'f_caranam')
('max_text_len', 1357)
('Part:', 'l_caranam')
('max_text_len', 397)


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

^C


In [3]:
import pandas as pd
from google.colab import files
uploaded = files.upload()
!ls

Saving div_songs.csv to div_songs.csv
datalab  div_songs.csv	div_songs.xls


In [0]:
lyrics_dataframe = pd.read_csv('div_songs.csv')['no_caranam']
print (np.mean(lyrics_dataframe))

3.026760563380282


In [0]:
# Creating 4 RNNs for pallavi, anupallavi, all but the last caranam and the last caranam
# input_size : size(vocabulary)+1 to account for the padding const
p_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
# rnn = torch.load('conditional_rnn.pth')
p_rnn.cuda()
ap_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
ap_rnn.cuda()
fc_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
fc_rnn.cuda()
lc_rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))
lc_rnn.cuda()

learning_rate = 0.001
p_optimizer = torch.optim.Adam(p_rnn.parameters(), lr=learning_rate)
ap_optimizer = torch.optim.Adam(ap_rnn.parameters(), lr=learning_rate)
fc_optimizer = torch.optim.Adam(fc_rnn.parameters(), lr=learning_rate)
lc_optimizer = torch.optim.Adam(lc_rnn.parameters(), lr=learning_rate)

# Loss function
criterion = torch.nn.CrossEntropyLoss().cuda()

def sample_from_rnn(starting_sting="", sample_length=200,sample_end="<EOP>", temperature=1, rnn=p_rnn, only_sample=0):
    ''' Sampling text from the RNN'''
    
    sampled_string = starting_sting
    hidden = None
    
    #Sending initial input as the starting string
    first_input = torch.LongTensor( string_to_labels(starting_sting)).cuda()
    first_input = first_input.unsqueeze(1)   # addds another dimension [1,2,3] -> [[1],[2],[3]]
    current_input = Variable(first_input)

    # Forward propogation
    output, hidden = rnn(current_input, [len(sampled_string)], hidden=hidden)
    
    # Get the last character in the output
    output = output[-1, :].unsqueeze(0)    # unsqueeze adds another dimension [1,2,3] -> [[1,2,3]]

    if rnn == lc_rnn and only_sample == 1:
        while (sampled_string.find("tyAga") == -1):
            sampled_string = starting_sting
            for i in xrange(sample_length):
        #  Softmax function gets values b/w 0 & 1 and the sum of the values in the tensor sum up to 1
        # basically gets the probabilities of each character
                output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data
                predicted_label = torch.multinomial(output_dist, 1)          # Multinomial distribution 1 -> number of samples to draw
                sampled_string += all_characters[int(predicted_label[0])]    # Get the next character
                current_input = Variable(predicted_label.unsqueeze(1)) 
                output, hidden = rnn(current_input, [1], hidden=hidden)      # Send the predicted character as input back into the rnn to get the next character
                if (sampled_string[-5:] == sample_end):
                  break
        return sampled_string

    for i in xrange(sample_length):
        #  Softmax function gets values b/w 0 & 1 and the sum of the values in the tensor sum up to 1
        # basically gets the probabilities of each character
        output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data
        predicted_label = torch.multinomial(output_dist, 1)          # Multinomial distribution 1 -> number of samples to draw
        sampled_string += all_characters[int(predicted_label[0])]    # Get the next character
        current_input = Variable(predicted_label.unsqueeze(1)) 
        output, hidden = rnn(current_input, [1], hidden=hidden)      # Send the predicted character as input back into the rnn to get the next character
        if (sampled_string[-5:] == sample_end):
            break
            
    if (sampled_string[-5:] != sample_end):
        sampled_string = sampled_string + " " + sample_end
    return sampled_string  

In [1]:
!pip install torch==0.4

    100% |████████████████████████████████| 484.0MB 22kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x561c7d30c000 @  0x7f7553b7e1c4 0x561c214470d8 0x561c21530d5d 0x561c2145a77a 0x561c2145f462 0x561c21457b3a 0x561c2145f82e 0x561c21457b3a 0x561c2145f82e 0x561c21457b3a 0x561c2145f82e 0x561c21457b3a 0x561c2145fe1f 0x561c21457b3a 0x561c2145f82e 0x561c21457b3a 0x561c2145f82e 0x561c2145f462 0x561c2145f462 0x561c21457b3a 0x561c2145fe1f 0x561c2145f462 0x561c21457b3a 0x561c2145fe1f 0x561c21457b3a 0x561c2145fe1f 0x561c21457b3a 0x561c2145f82e 0x561c21457b3a 0x561c2148850f 0x561c21483202


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# %matplotlib notebook

# from matplotlib import pyplot as plt

# # importing a list that can be binded with a figure and update
# # the figure when being appended
# from visualization import VizList

# # Creating figure, axes and binding to lists 
# f, (loss_axis, validation_axis, train_axis) = plt.subplots(3, 1)

# loss_axis.plot([], [])
# validation_axis.plot([], [])
# train_axis.plot([], [])


# loss_list = VizList()
# validation_list = VizList()
# train_list = VizList()

# loss_list.bind_to_axis(loss_axis)
# validation_list.bind_to_axis(validation_axis)
# train_list.bind_to_axis(train_axis)

# loss_axis.set_title('Training Loss')
# validation_axis.set_title('Validation Set Accuracy')
# train_axis.set_title('Training Set Accuracy')


# plt.tight_layout()

In [0]:
files.download('conditional_rnn.pth')

In [6]:
clip = 1.0
epochs_number = 1000

for epoch_number in xrange(epochs_number):

    print ("epoch_number", epoch_number)
    for p_batch in p_trainset_loader:
#         print ("batch.shape", batch[0].shape, batch[1].shape, batch[2].shape)
        
        post_processed_batch_tuple = post_process_sequence_batch(p_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        p_optimizer.zero_grad()
        p_logits, _ = p_rnn(input_sequences_batch_var, sequences_lengths)
        p_loss = criterion(p_logits, output_sequences_batch_var)
        p_loss.backward()
        p_optimizer.step()
        
    for ap_batch in ap_trainset_loader:
        post_processed_batch_tuple = post_process_sequence_batch(ap_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        ap_optimizer.zero_grad()
        ap_logits, _ = ap_rnn(input_sequences_batch_var, sequences_lengths)
        ap_loss = criterion(ap_logits, output_sequences_batch_var)
        ap_loss.backward()
        ap_optimizer.step()
        
    for fc_batch in fc_trainset_loader:
        post_processed_batch_tuple = post_process_sequence_batch(fc_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        fc_optimizer.zero_grad()
        fc_logits, _ = fc_rnn(input_sequences_batch_var, sequences_lengths)
        fc_loss = criterion(fc_logits, output_sequences_batch_var)
        fc_loss.backward()
        fc_optimizer.step()
        
    for lc_batch in lc_trainset_loader:
        post_processed_batch_tuple = post_process_sequence_batch(lc_batch)
        input_sequences_batch, output_sequences_batch, sequences_lengths = post_processed_batch_tuple
        output_sequences_batch_var =  Variable( output_sequences_batch.contiguous().view(-1).cuda() )
        input_sequences_batch_var = Variable( input_sequences_batch.cuda() )
        lc_optimizer.zero_grad()
        lc_logits, _ = lc_rnn(input_sequences_batch_var, sequences_lengths)
        lc_loss = criterion(lc_logits, output_sequences_batch_var)
        lc_loss.backward()
        lc_optimizer.step()

        #torch.nn.utils.clip_grad_norm(rnn.parameters(), clip)

    # Getting the number of caranams
    no_caranam = max(1, int(np.ceil(np.random.normal(caranam_mean, caranam_std))))
    
    # Sampling pallavi
    print(sample_from_rnn(rnn=p_rnn, starting_sting="pallavi\n"))
    
    # Sampling anupallavi
    if np.random.normal(ap_mean, ap_std) > ap_mean:
      print(sample_from_rnn(rnn=ap_rnn, starting_sting="anupallavi\n"))
    
    # Sampling first_caraNam
    for i in range(no_caranam-1):
      print(sample_from_rnn(rnn=fc_rnn, starting_sting="caraNam " + str(i+1) + "\n"))
    # Sampling last caraNam
    if (epoch_number >= 800):
      if no_caranam == 1:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>", only_sample=1))
      else:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>", only_sample=1))
    else:
      if no_caranam == 1:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>"))
      else:
        print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>"))
      
    torch.save({'prnn':p_rnn.state_dict(), 'aprnn':ap_rnn.state_dict(), 'fcrnn':fc_rnn.state_dict(), 'lcrnn':lc_rnn.state_dict()}, 'conditional_rnn.pth')
#     torch.save(rnn.state_dict(), 'conditional_rnn.pth')
    
    

('epoch_number', 0)


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


pallavi
 aalva EPr>Pv

Ohira mamAlOturanA >Og>va
cl<PO>
av
cnvEi  aa nnAa mp mainOtirnDhrE 
EE
Ajrm
AmesijrE vA ntuua EAa makAlNaAulinamrmAI OPIiAaka
kc >vailra p
di >usr dai DtjamaDtlrAulamAnaTia a Ar  upadl <EOP>
caraNam
DmlAm<btaaanh i  ya lvreyATIass
a>daAaj>  mrjbStaoEnaaaaArarSa hu>r  jA
AIhuamuukun smana a>gaiua san ha nma nA <akahtanua ykS duaibgrur
abaOarau  avAu>

un
 hh<hrOga ytit
n Ag  aama vAaunA Aam  ia Sn <EOS>
('epoch_number', 1)
pallavi
inEnbusE ymea Eanasvu uhavirA jAti raenla amkana vTuu lallkP mamE jamga lana kaladEI na ealui mAuNmarAnanu danbmu <EO>
>r
da EmA hanhaliyu huna da <EEOP
>

vA masayhEka kAmAla ruAA puna sAvi lOPli sEu <EOP>
anupallavi
da yuhrha horlnENaviO
iausapa ulaa haa uEEdr <PArATanakukjlanAusavg pakaTualalla  gspcanOlvA
inavva t
hrANaraSahyu a ndADauudpava ta OankT pi indu OhAEShmlativunurva pOdmDama SI btAkirar sA akanhAra < <EOP>
caraNam 1
dymac
nia h  honrajnaDsmu ruI  aeaTN aalh atya nkaiuna alcabTiP dtAaEmEjaOii Prad
s Eineuy uNny ruvanCuaN l

anupallavi
Asa nI mUNdu tumanau karamu bItacunabhaNamcu vAtrai dpajaja reTciSi celakuu melrbha bATamu <EOP>
caraNam
madaDayi peja NInu cencisiTi cinni gaguNanu tyAgarAjalO sEdaT-varita vATArpu bhAgAhAri <EOS>
('epoch_number', 8)
pallavi
kugui nA parimpItAkO deliyAdvaram O <EOP>
caraNam 1
<EOP>
caraNam 2
tyAgarAjamucu koDagu bOdundhi uldambuN gannu bAkma pamagaa lEki baliti shrI cESa karaNam
varakurAma <EOS>
('epoch_number', 9)
pallavi
gAtarEm Ujupara mAyam? <EOP>
caraNam 1
dhOviraSara sIti nIrA kE <EOP>
caraNam 2
tan<EOP>
caraNam 3
nittaka bhUSaNamu tyAgarAjuni bhUpa <EOS>
('epoch_number', 10)
pallavi
vidciDa jUDu O rAma rAma galamu <EOP>
caraNam 1
citulana hUda ulci maktusanna samulata sriva atApaDagga manranu <EOP>
caraNam 2
ndadhabhajyu maTkana morita Ti nEka unta mOvuna naya saTka bUra santula nAkE shiva yAntamE rAsharanlu bummulugu mEibhapa jiti bhAramEDa egparamu songanambunelu ranu rIyu vIrhSalaina <EOP>
caraNam 3
dAnvamanE vani somanucu tALa nincanu guTTi nicni belri rAjAnjamul

pallavi
mA pamma nandanakan- <EOP>
anupallavi
moralucu PimamA <EOP>
caraNam 1
sangananu garamA 
sadAla rAgana
sambha paramA nALu tanaDu nIma <EOP>
caraNam 2
dAlatETaDu pOvadanU prammmamA ramunu manaTEcanpu jUcanrasAra <EOP>
caraNam 3
gala muNcu nimmati nanganu tana golitA rAdhishinci virbhuvarAdrakanu AsancAra bhAmulu
gati <EOP>
caraNam 4
dUSa dErada manupagucu
para
kOtaka rAkya ganimA para lUpa sukha bhatya pAda shrI <EOP>
caraNam 5
ganulaku lellaku tyAgarAja vara gonuvu sAkSikha pADa 
novimAdanu tyAgarAja prEmuna <EOS>
('epoch_number', 21)
pallavi
dayarAham trada niladu koda pUja pOcEdIrha rUpa tryatamma harUcra <EOP>
caraNam 1
pAgaTamu mavya 
pelDuku mennucu
sAru dUrm nandamu <EOP>
caraNam 2
Samanmulu jAtukonnu <EOP>
caraNam 3
jagamuna vinutvumu lEdA lAkumA alukupoka naTyya
kapincalA tyAgarAja hrdayana gAikaTiti visai brOvagA <EOS>
('epoch_number', 22)
pallavi
upuya phaTTa bAlagaNTini dendugAna pUrTAcanu rakSincA valenO <EOP>
caraNam 1
nodaku mari shOjAdru mAnitO sa viNna shrAsamu m

...

caraNam 3
tAraka jUci tA kAryE pughuna dayAlEka rAjarAya 
shara dhAna nijanamu jEyega sukhamu shUlunaina janulu mari brOtuvayya <EOP>
caraNam 4
tam mIlOham mAku tajtuvadya sadanta <EOP>
caraNam 5
tAralu nuppa navvurAye calanAduru guriyani balkirAla yugamuna jElagaTTu ika karmamuna nIkani saragituDA jananE brahmAnguni dinamunu shESa nIlanka samAtra vipuNTalADa rOra brOcu vADana bhArajita vinuta <EOP>
caraNam 6
rAdA rahrta sarva nyAyamu karuNAlita sallApamuna nAvE <EOP>
caraNam 7
kanuNDIdi muni karuraNam sItAnAtma shrI tyAgarAjuni bhAma janakAdulaku <EOS>
('epoch_number', 491)
pallavi
pAhi rAma dUta jagatprANa kumAra mAm <EOP>
caraNam
dEgakai mEnOsangu rAghavAnvayamuna bAga janmincina tyAgarAjanuta shrI <EOS>
('epoch_number', 492)
pallavi
viDamu sEyavE nannu viDa nADakavE <EOP>
anupallavi
mArakOTi saundaryamunu gani madilOna sunta maraciyuNTE <EOP>
caraNam 1
sEva jEsina konna rakSinci mencanu nEn- <EOP>
caraNam 2
cana varama karurArAdi bhata jagadA manasu antanE vidya
cApamE janigonaga k

caraNam 2
taramu bOnE aDaku nE verave terukukOra tOri trakSamu dhanatAravimpaga kamanOhara sharatAmrnayya <EOP>
caraNam 3
jupEraNam asurA nIla kAbhamula tOnu nEra putraghO <EOP>
caraNam 4
hari sairevaru shrI tyAgarAja varaduni pogaDucu <EOS>
('epoch_number', 498)
pallavi
kaNTakulanu nammanEranurA mukkaNTi sannuta shrI rAma
puNTi vaNTi dAsavenTa tIrcETaTuvaNTi daivamu nIviTa nuNDaganE <EOP>
anupallavi
shambaravairijanaka sOdari sharaNu jocci manasAra shrI jaga <EOP>
caraNam 1
vanamuna delisi yAdu gOppAdA nA muktula rAyamA rAvE <EOP>
caraNam 2
tAraka sugalArja su-bhUSa bhavya sEvita sagulu mangaLam E bhakSyamellanu <EOP>
caraNam 3
nAgarAja shayana vidhIndrA bhava sAgarOttaraNa rAma sAratara shrI tyAgarjanuta caraNa <EOS>
('epoch_number', 499)
pallavi
nAdasudhA rasambilanu narAkrti yAyE manasA <EOP>
anupallavi
para bhAmala nAshinci annamiDi pagalu rEyu sarasa mADu vAri nolla <EOP>
caraNam 1
tiruga bhava pAdamanaru lEDu nA sukhamu <EOP>
caraNam 2
naravara bAdha nI sannitAngi ninnA
karuNa i

caraNam 3
dhIra nAdulakana yavArivairi nI manakhulu sughye munindu vEDuka nataja nava viuchunAtha kunnavAralu sundara pAlana
vidAra sava bhaktiyandu vArginci tapamantulu sarigani kaDa tOriyaina surAreku kaSTamu <EOP>
caraNam 4
mAnamu cEya gItAdyamara ishAlaru surapati kEgincitE <EOP>
caraNam 5
ciracamadu lakSyamu kalisitE nannu sannuti sItAvariki
tAnurakAntamatru vIda kala kAlamu gAdA <EOP>
caraNam 6
sukha mosagi bhavamukha kara maravanu rAma <EOP>
caraNam 7
jikti ani bhakta pAlita tyAgarAja bhAgyamA <EOS>
('epoch_number', 509)
pallavi
sugaramu pogaDa taramA n tAlincina pUjE pogaDaDugu mukhADi <EOP>
anupallavi
mOhana rAma moraliDagA karuNa lEka nIvE nanu jUDaga <EOP>
caraNam 1
sAdhu mAraka ghanamuna svAnta suku-ratsamUta <EOP>
caraNam 2
tAraka guNa kArya mAnavulai gAvalace nI bimmiti hipadEla bhuvini tAkanI ramaNuDe gAvinci tanuvunu vElavE <EOP>
caraNam 3
jita manOrathulaku mauna yatula tapO niyatulakai hitamu sEyaka hitapu vratalanu jEndina
shrta padambu jIvanAmrta phalambu nIvani nA 

Process Process-8265:
Process Process-8267:
Process Process-8266:
Traceback (most recent call last):
Process Process-8268:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
    self.run()
    self.run()
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self.run()
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self

KeyboardInterrupt: ignored

In [7]:
    ''' Sampling '''
    for i in range(1000):
        print ("example", i)
        # Getting the number of caranams
        no_caranam = max(1, int(np.ceil(np.random.normal(caranam_mean, caranam_std))))
    
        # Sampling pallavi
        print(sample_from_rnn(rnn=p_rnn, starting_sting="pallavi\n"))
    
        # Sampling anupallavi
        if np.random.normal(ap_mean, ap_std) > ap_mean:
            print(sample_from_rnn(rnn=ap_rnn, starting_sting="anupallavi\n"))
    
        # Sampling first_caraNam
        for i in range(no_caranam-1):
            print(sample_from_rnn(rnn=fc_rnn, starting_sting="caraNam " + str(i+1) + "\n"))
        # Sampling last caraNam
        if (epoch_number >= 800):
            if no_caranam == 1:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>", only_sample=1))
            else:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>", only_sample=1))
        else:
            if no_caranam == 1:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam\n", sample_end="<EOS>"))
            else:
                print(sample_from_rnn(rnn=lc_rnn, starting_sting="caraNam "+ str(no_caranam) +"\n", sample_end="<EOS>"))
        print("\n\n")

('example', 0)
pallavi
pAhi ramAramaNa mAm pAhi sadguNa gaNa harE mAm <EOP>


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:55: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


caraNam 1
cEta cadula vAra delisi cElavaiyuNDagA nA shrI rAma <EOP>
caraNam 2
vAraina paripAlani blivuDu nIkika ninnASalamunanu muddu vEnka nI pada lEdanu drSTambulu
naDaci anniTa nAye gatiyA brarAvaNuni gaTTi
paratOdurE naludunoka pactalam amba jAnaka janaka vinOharu lErO tETa <EOP>
caraNam 3
dhyAnini balkina rAdu prapanca nrpuDu madinAcE brahdhu kushimukhula jArci <EOP>
caraNam 4
nE nanatO pArijAta su-patiki nIla hita kulAsamu sEya tyAgarADi inta cittamanuta kAntaka kAvavE <EOP>
caraNam 5
sAkAri shashimukha nI kevarE sAyega <EOP>
caraNam 6
jita manOrathulaku mauna yatula tapO niyatulakai hitamu sEyaka hitapu vratalanu jEndina
shrta padambu jIvanAmrta phalambu nIvani nA hitamu pUja viDa nADina ataDE tyAgarAjanuta <EOS>



('example', 1)
pallavi
vidhiyEmi jEyudurA pashupati nIvuNDaga <EOP>
caraNam 1
vamEna pAlana paripAla pUrita guNamu laruguNa nilanAgha kumudhalunin munu
tOnu nanna priyaputri putriti cEntella nA kaDuNDunu jEsina nE bhuvini pUjincucu <EOP>
caraNam 2
vara bhamO aTa lIDu

caraNam 1
vanamana nIkulaityA sAkSasApta hrdaya vEdana ghAna moravevari galu gambhIram <EOP>
caraNam 2
vAraNa rEkumOTa kushalajavairi O rAghava rAma pApayuni balkitE nI mahimaDu tangu nIva nE nE velasi nI lEni <EOP>
caraNam 3
dhurambuga olla rAjanya bOyi poTTaga jagamatO sannigAkanIda nApAlanE <EOP>
caraNam 4
ninnAnA garAjulaku nI dayAdula lEni jAli jEsina dAna mElu hammi kaliga gala-
bjUci deliya vEmulaina E phalamA <EOP>
caraNam 5
kAmi tIra bhUSa bhacatam vairi lAvaNya supradEshi nE sheyAryamu nIvEdu <EOP>
caraNam 6
cApa sAgarIsha bhayamauinthE n Arpala mOkSa madhAra shrI rAma kala dharaNina murasan-
tharaNa ankita shApyamu rajnArIpadi nE shrIsha jiktitO kAla naTiyanu rAvu <EOP>
caraNam 7
vanana jana priya Anubhava phala mAna sutarO lanucu psadalundu tana kanikonusava matini kanu kani nelabAkhi dasharatha nandana <EOP>
caraNam 8
kAsjincu mATala vrAla karuNatO sanndhilAnamadaga karamulu sAyaka nI pAlamella jEsina gOri konnaDAka rAma nannya ghana maracitE jEsinAnO <EOP>
caraNam 9
harir

caraNam 4
jimvara parAshara nArada hrt-pankERuha nIraja bandhO
ahlAda kara bhavarOga samhAra varada tyAgarAjAdi vinuta <EOS>



('example', 18)
pallavi
muddumOmu ElAgu celangeno munulETla gani mOhincirO <EOP>
anupallavi
dAnta sItAkAnta karuNAsvAnta prEmadulakE delusunu gAni <EOP>
caraNam
dharanu tyAgarAja vinutuni palukulu tappi bOdana lEdA <EOS>



('example', 19)
pallavi
rAma nAmamu janma rakSaka mantramu tAmasamu sEyaka bhajimpavE manasA <EOP>
anupallavi
centajEri saujanyuDai baliki cinta bAga tolaginci brOcitivi <EOP>
caraNam 1
vammita vibhakta pApa tini
bhIma bhAgavatulu tama priyE pacAla hrdayArnava phaladula ginnalOku mAga rathamulanu 
gAcina gAcina prEmahara ninu cintarAdu adigAka dAsurA dEha sharO-bhAvanamunu dayatO cEn <EOP>
caraNam 2
jAgaramunaku modaTi daivamu nI shakti sumukhata bhaktulagu vAriki
siddhi buddhi vara phalamu nosagina sadguru shrI tyAgarAju pogaDina <EOS>



('example', 20)
pallavi
tava dAsOham tava dAsOham tava dAsOham tava dAsOham dAsharathE <EOP>
caraNam
v

...

caraNam 3
nityambu ninu krSNa vancana jEsina cakka sUrya sanakAdi vinuta nE dhanamu <EOP>
caraNam 4
mAnapai tyAgarAja vihArA gata kAmita phAri duSTa dUra tanaya capalEkAyE manasEi dharma delisi <EOP>
caraNam 5
guTTu kannau gaTTunaka nannulAra madamukha vadana kara kanakAbha tanakayuta guNAntamuna nelakitinO <EOP>
caraNam 6
manasuna su rAsula brOcinadAlO
bhajiyimpa sEva lElEni gati tyAgarAjanuta <EOS>



('example', 950)
pallavi
cani tODi tEvE O manasA <EOP>
anupallavi
karthAmrtamucE bahukAlamu akali dIriyunnanu brOvumu <EOP>
caraNam 1
nEtara dhana sajjana lOcana nIdu vacana sItApati bahktula renceTTO bAgutuDaina satajana nI <EOP>
caraNam 2
vanana duSTa jana gamana mAdhava vimala nA bhUnita guNa bhakta pAvana najana sEga deivamu gAna nI vaddhE <EOP>
caraNam 3
muni maracitinArA O gana <EOP>
caraNam 4
mEnu vAnini jUci nAyaku pannula kUla jEya nInuDai sajjaurAya <EOP>
caraNam 5
jalanjulEka sharaNAsanuni kanna pinnaga jagamuniki 
parama nIku duruni jEsina duSkarmamula  nanaganu sEya nAru sa

pallavi
enduku nirdaya evarunnArurA <EOP>
anupallavi
pankajAkSa nIvaNTi parama dayAnidhi nI <EOP>
caraNam
panasa nArikElAdi jambU phalamula nAraginci ghana tarambagu mahipai padamulu 
ghallu ghallana nunci anyamu hari caraNa yugamulanu hrdayAmbujamuna nunci 
vinayamunanu tygarAja vinutuDu vividha gatula d <EOS>



('example', 961)
pallavi
rAmAbhirAma raghurAma O rAma <EOP>
anupallavi
sammatiga rAju koDukanaga vinisamshayamu dIrakashrI rAmuni <EOP>
caraNam 1
koncEpu guNamula kAsinci tA krSNa rasa munaku grugamuna
nEn- <EOP>
caraNam 2
naradautaku Emayakhara sarOjalEka bharatuni jEsi nEDu bOlu vadanumA marma mOmuna barada layamu rAmayA balimi nela tallitOvva lEka 
teliya nI kAshincina cittula nosagucu nunnAru <EOP>
caraNam 3
dhAri vishAlayula jUci pApala vinutu santatam evaru navvugAnu pAla
suraTicE nugha vincAlya muni jana yAgyamulE nI <EOP>
caraNam 4
ninu pAtramu nI jocinu teliyakayu taLaDu telisi niNDi <EOP>
caraNam 5
bharatu prrayabAdhAri taurituDati mOva beTu kappunna cintanu mura vi

anupallavi
ilanantaTa tirugucunu galuvarincErE gAni <EOP>
caraNam
agaNita guNa kanaka cEla sAla viDalanAruNAbha samAna caraNApAra
mahimAdbhuta su-kavijana hrtsadana sura munigaNa vihita kalasha nIra
nidhijA ramaNa pApa gaja nrsimha vara tyAgarAjAdhinuta <EOS>



('example', 971)
pallavi
tIrunA lOni dugdha <EOP>
anupallavi
bhaktavatsala patita pAvana trishaktiyu galgina dEvuDu nIvE <EOP>
caraNam 1
mEdi tumburu nAradAdula dayADanAye ni nE svambhASaNa nArtra khaNyauna <EOP>
caraNam 2
parAkEmaina ceppakurA kEkalu vEturu rA kEshava shrI tyAgarAjanuta shubha carita <EOS>



('example', 972)
pallavi
ennaDO rakSincitE nI vipuDIvaganu uNDavalenA <EOP>
anupallavi
trippaTalanu dIrci kaNTi reppavalEnu gAcu mA <EOP>
caraNam 1
vanamAnanya nibhacAra kanaka saguNa varada <EOP>
caraNam 2
sAdhu mAta pAlita padvini sadA nI panulEla I kAsurA gatyamulu <EOP>
caraNam 3
kushAluda kula kAshincu ghana janakOTina bhaktula kalAyamunu <EOP>
caraNam 4
mElanu tA pulakariyu vEga bOtina vEpulanu vEga nIveta kOkiva sa

caraNam 1
vananAbha nAdamulamunu bhayamEDu jnAnOTlaku nI jutikina pUjA phalamO lEka nIdu gangAmara mAtamulanu <EOP>
caraNam 2
vAmara nindukA nEnaina pAlincina enta nIru vandarpUrS kODlE karamuna merayu gramma ninnugaTTa ramrakSaNamElE pANimita rArA <EOP>
caraNam 3
nAmamu mudamau rOvi gAri gati nI kAvalini sOvasanamA nirjita suganma <EOP>
caraNam 4
 vEsuni bOdu rA rAma nI dAsa ollavalalO rAma <EOP>
caraNam 5
jagatvarUpa sukhamA 
sharaNaga oka mATaku vinhISaNuDu OrvalEka sharaNaga 
sakalEshvara prahlAduDu jAlicE sharanaga hita karuNaDai brOcitivE tyAgarAjuni mATa <EOS>



('example', 984)
pallavi
sundari ninu varNimpa brahmAdi surulakaina taramA <EOP>
anupallavi
nIkE marulaitini nIkEla I guNamu <EOP>
caraNam 1
cEta tArakala rAma krta nipai bhrama sEya daTa brOva <EOP>
caraNam 2
vadaci tanamu kaliga jEsina sutudu sampadamu sEyu vETalayA jEyu nI vAru vAyu lEdaNDala nunnAnu darmam dusnEnu tA vEDukonna nIku dAsi Eruganu <EOP>
caraNam 3
nEtajitta kOTi samhAra nEtram pApi bUnikai yAshrita anay

caraNam 3
dhArini dIrEka tana kIrti vIlukai mE nEmi eDabAya nA sandhAntuDai tErOca nEnu nA <EOP>
caraNam 4
tAdurA tanaya karuNAkara bhUsharaNa maniri dEhAdhIra 
sarasA-nataja bharAtbha kumnAra pAlatAlambu cEyu suma mA mEla 
mAyala nibhavENam <EOP>
caraNam 5
dEha mEkula jUturA kulambutO pongiDitirA dAlamu nIdya tyAgarAja shayana nIkinci tA jAli delisi
malammativE ninnAralEdA nI dEvayani santOSamu ati nEramO teliya <EOP>
caraNam 6
mAra jaladhara traja prahlitu rAni dUri cIturE nI padamuna
teliyani ninnu goluvadu shrI ArNina nIku yuga balkaga nE manasu dIrpa lEdu kAvatuDayu tapamE nishcapati nIlavarNa pura nITa mAdhavuniki suguN <EOP>
caraNam 7
nEtruDaina santatamu saregA  shiskamuna tirigina cOcenO <EOP>
caraNam 8
manasu ranjilla peddalatOnu kalla lADaka molla sumamula 
nI callani padamula golla lADucu vEda callitigAni tyAgarAjunipai nIk- <EOS>



('example', 994)
pallavi
yajnAdulu sukhamanu vAriki samula jnAnulu gAvalenayya <EOP>
caraNam 1
vatsalam shivamukhAyuni dEvanIshula cakkani mOs

In [0]:
# !ls
# from google.colab import files
# files.download('conditional_rnn.pth')
a = torch.tensor([1,2,3])
b = a.split(split_size=1)
# output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data
# predicted_label = torch.multinomial(output_dist, 1)


[(1, 4, 7), (2, 5, 8), (3, 6, 9)]